# Neural networks - Project 1 
## Luis Filipe Menezes - RA: 164924

Este notebook foi feito como entrega para o 1º projeto da disciplina de Redes Neurais do Programa de Pós Gradução em Ciência da Computação (PPG-CC) da Univesidade Federal de São Paulo (UNIFESP).

Este projeto consiste em:

- Selecionar dois datasets (não triviais)

    - Um dataset para **classificação**

    - Um dataset para **regressão**

- Separar em treino/validação/teste

- Treinar modelos MLP para os dois problemas

- Considerar:

    - Diferentes topologias (>=5 topologias, variar número de camadas)
    - Usar o algoritmo original SGD (não usar algoritmos otimizados, e.g.ADAM)
    - Avaliar o impacto do uso do Momentum
    - Avaliar o impacto do uso da regularização (i.e. L2)

- Ilustrar graficamente a evolução do treinamento (treino/validação).

- Confeccionar um relatório (reprodutível) contendo os experimentos e resultados


# Predicting Crocodiles' conservation status

O primeiro problema a ser resolvido consiste em um problema de classificação utilizando um *dataset* 
sobre conservação de espécies de crocodilos.

## Upload do dataset

Faremos o upload do *dataset* pela biblioteca **KaggleHub**. 


In [4]:
import sys
import subprocess
import pkg_resources

try:
    pkg_resources.get_distribution('kagglehub')
    print("A biblioteca kagglehub já está instalada.")
except pkg_resources.DistributionNotFound:
    print("A biblioteca kagglehub não foi encontrada. Instalando...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "kagglehub"])
    print("kagglehub instalado com sucesso.")

A biblioteca kagglehub já está instalada.


In [12]:
import kagglehub
import pandas as pd

# Download latest version
def download_and_load_dataset():
    path = kagglehub.dataset_download("zadafiyabhrami/global-crocodile-species-dataset") + "/crocodile_dataset.csv"
    df = pd.read_csv(path)
    return df, path

if __name__ == "__main__":
    df, path = download_and_load_dataset()
    print("Dataset preview: \n", df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Observation ID        1000 non-null   int64  
 1   Common Name           1000 non-null   object 
 2   Scientific Name       1000 non-null   object 
 3   Family                1000 non-null   object 
 4   Genus                 1000 non-null   object 
 5   Observed Length (m)   1000 non-null   float64
 6   Observed Weight (kg)  1000 non-null   float64
 7   Age Class             1000 non-null   object 
 8   Sex                   1000 non-null   object 
 9   Date of Observation   1000 non-null   object 
 10  Country/Region        1000 non-null   object 
 11  Habitat Type          1000 non-null   object 
 12  Conservation Status   1000 non-null   object 
 13  Observer Name         1000 non-null   object 
 14  Notes                 1000 non-null   object 
dtypes: float64(2), int64(1

## Preparação de dados

Existem 15 atributos no total, no entanto, nem todos podem ser utilizados para classificação, e.g. *Common Name*, *Notes* ou *Family*.

In [14]:
cleaned_df = df.drop(columns=['Common Name', 'Notes', 'Family', 
                              'Observer Name', 'Country/Region', 
                              'Date of Observation', 'Sex'])

In [16]:

cleaned_df = df.drop(columns=['Common Name', 'Notes', 'Family', 
                              'Observer Name', 'Country/Region', 
                              'Date of Observation', 'Sex'])

# Célula para inspecionar as colunas categóricas
categorical_cols = cleaned_df.select_dtypes(include=['object']).columns
print("Colunas categóricas restantes:\n", categorical_cols)
print("\nNúmero de valores únicos por coluna:")
print(cleaned_df[categorical_cols].nunique())

Colunas categóricas restantes:
 Index(['Scientific Name', 'Genus', 'Age Class', 'Habitat Type',
       'Conservation Status'],
      dtype='object')

Número de valores únicos por coluna:
Scientific Name        18
Genus                   3
Age Class               4
Habitat Type           29
Conservation Status     5
dtype: int64


In [19]:

# Identifica as colunas categóricas para codificar
# Supondo que 'Scientific Name' e 'IUCN Red List Category' sejam as principais
cols_to_encode = ['Age Class', 'Genus', 'Conservation Status'] # Ajuste conforme sua análise

# Aplica o One-Hot Encoding
encoded_df = pd.get_dummies(cleaned_df, columns=cols_to_encode, drop_first=True)

print("Dimensões do DataFrame após One-Hot Encoding:", encoded_df.shape)
encoded_df.head()

Dimensões do DataFrame após One-Hot Encoding: (1000, 14)


,Observation ID,Scientific Name,Observed Length (m),Observed Weight (kg),Habitat Type,Age Class_Hatchling,Age Class_Juvenile,Age Class_Subadult,Genus_Mecistops,Genus_Osteolaemus,Conservation Status_Data Deficient,Conservation Status_Endangered,Conservation Status_Least Concern,Conservation Status_Vulnerable
0,1,Crocodylus moreletii,1.90,62.0,Swamps,False,False,False,False,False,False,False,True,False
1,2,Crocodylus acutus,4.09,334.5,Mangroves,False,False,False,False,False,False,False,False,True
2,3,Crocodylus intermedius,1.08,118.2,Flooded Savannas,False,True,False,False,False,False,False,False,False
3,4,Crocodylus moreletii,2.42,90.4,Rivers,False,False,False,False,False,False,False,True,False
4,5,Crocodylus palustris,3.75,269.4,Rivers,False,False,False,False,False,False,False,False,True
